In [ ]:
# import sys
# from  ift6758.data.data_acquisition import Season
# season2018 = Season(2018,"../ift6758/data")
# df_season2018_clean = season2018.clean_data()
# df[["gamePk","liveData.plays.allPlays"]]

In [ ]:
from ipywidgets import interact, RadioButtons, IntSlider, Output, Layout, Dropdown
import plotly.graph_objects as go

import json
import pandas
import numpy as np
from tabulate import tabulate
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns

import requests
import json

In [ ]:
IMG_PATH = "../figures/nhl_rink.png"

In [ ]:
url = 'https://statsapi.web.nhl.com/api/v1/teams'

r = requests.get(url, headers={'accept': 'application/json'})
json_teams = r.json()

print(json_teams["teams"])

In [ ]:
team_list = list()
for team in json_teams["teams"]:
    team_list.append(team["name"])
print(team_list)

In [ ]:
#CHOOSE GAME ID
season_selector = IntSlider(
    value=2016,
    min=2016,
    max=2020,
    step=1,
    description='Season: ',
    continuous_update=False, #only reassign value when the slider stops moving
    layout=Layout(width='50%')
)

#CHOOSE GAME TYPE
team_selector = Dropdown(
    options=team_list,
    value='New Jersey Devils',
    description='Number:',
    disabled=False,
    continuous_update=False,
    layout=Layout(width='50%')
)

In [ ]:
def fig_shot_map(team, season):
    
    #create figure
    img = plt.imread(IMG_PATH)
    fig, ax = plt.subplots(figsize=(10, 5))
    
    #prep the ring and the axis of the ring
    plt.xlim(-100, 100)
    plt.ylim(-42.5, 42.5)
    plt.xlabel("feet")
    plt.ylabel("feet")
    plt.title(team + '\n' + 
              "Season: " + str(season) + "  Relative to League Average for the Season ")
    ax.imshow(img, extent=[-100, 100, -42.5, 42.5], aspect='auto')
    
    #get random data
    from random import randint
    delta = 0.5
    x= np.arange(-100, 100, delta)
    y = np.arange(-42.5, 42.5, delta)
    X, Y = np.meshgrid(x, y)
    Z1 = np.exp(-X**2 - Y**2)
    Z2 = np.exp(-(X - 1)**2 - (Y - 1)**2)
    #Z = (Z1 - Z2) * 2
    Z= X - Y
    Z = Z.reshape((len(y), len(x)))
    print(X)
    print(Y)
    print(Z)
    
    #create cmap
    norm = cm.colors.Normalize(vmax=abs(Z).max(), vmin=-abs(Z).max())
    cmap = cm.PRGn
    
    #draw the contour maps
    levels = np.arange(-1.2, 1.6, 0.2)
    
    CS1 = ax.contourf(X, Y, Z, levels, norm=norm,
                   cmap=cm.get_cmap(cmap, len(levels) - 1),
                   extent=[-100, 100, -42.5, 42.5],
                    linewidths=2
                  )
# It is not necessary, but for the colormap, we need only the
# number of levels minus 1.  To avoid discretization error, use
# either this number or a large number such as the default (256).

# If we want lines as well as filled regions, we need to call
# contour separately; don't try to change the edgecolor or edgewidth
# of the polygons in the collections returned by contourf.
# Use levels output from previous call to guarantee they are the same.

    CS2 = ax.contour(X, Y, Z, CS1.levels, colors='k')
    
    # make the color bar for the contour lines
    CB = fig.colorbar(CS1, shrink=0.8)
    #change its location
#     l, b, w, h = ax.get_position().bounds
#     ll, bb, ww, hh = CB.ax.get_position().bounds
#     CB.ax.set_position([ll, b + 0.1*h, ww, h*0.8])
       
    #show it
    plt.show()

In [ ]:
# import plotly.express as px
# from skimage import io
# def fig_shot_map(team, season):
    
#     #create figure
#     img = plt.imread(IMG_PATH)
    
#     img = io.imread('https://upload.wikimedia.org/wikipedia/commons/thumb/0/00/Crab_Nebula.jpg/240px-Crab_Nebula.jpg')
#     fig = px.imshow(img)
#     fig.show()


In [ ]:
#display both widgets
display(season_selector, team_selector)

#init season as 2016
season = "2016"

#LINK season to widget 
def update_season(change):
    global season
    season = change['new']
    
season_selector.observe(update_season, 'value')

#init team
team = "New Jersey Devils"

output = Output()
display(output)

#LINK team to widget 
def update_team(change):
    
    #update team
    global team
    team = change['new']
    
    fig_shot_map(team, season)
    
team_selector.observe(update_team, 'value')